In [1]:
import os
import sys 
os.chdir("/workspaces/dev/modules")
sys.path.append("/workspaces/dev/models/R-CNN")

In [2]:
from tensorflow import keras
import tensorflow as tf

2025-03-10 04:58:12.127422: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-10 04:58:12.138209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741582692.149543   39218 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741582692.152673   39218 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 04:58:12.164025: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
from Pills.GenerativeSequence import GenerativeSequence
from PillNet import PillNet

In [4]:
SOURCE_PATH = '/workspaces/dev/datasets/pills/data'
LABEL_PATH = '/workspaces/dev/datasets/pills/class_label.csv'
BACKGROUND_IMAGEES_PATH = '/workspaces/dev/data'

In [5]:
TRAIN_DATA_SIZE = 60000
VALIDATION_DATA_SIZE = 20000
BATCH_SIZE = 16
INPUT_SIZE = (128, 128, 3)

In [6]:
model = PillNet(BATCH_SIZE, 50)
model.summary()

I0000 00:00:1741582694.400117   39218 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9502 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "pill_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ FeatureExtractor (Sequential)   │ (None, 32, 32, 32)     │       130,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ROIHead (Sequential)            │ (None, 128, 128, 1)    │         5,793 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DetectionHead (Sequential)      │ (None, 16, 4)          │     3,250,564 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ CentroidHead (Sequential)       │ (None, 2)              │        23,210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SegmentationHead (Sequential)   │ (None, 32, 32, 1)      │        37,913 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ClassificationHead (Sequential) │ (None, 50)             │     1,367,410 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,815,050 (18.37 MB)

 Trainable params: 4,815,050 (18.37 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
train_sequence = GenerativeSequence(
  TRAIN_DATA_SIZE, BATCH_SIZE, input_size = INPUT_SIZE[:2],material_path=SOURCE_PATH,
  label_path=LABEL_PATH, background_images_path=BACKGROUND_IMAGEES_PATH,
  use_multiprocessing=True, workers = 12
)
validation_sequence = GenerativeSequence(
  VALIDATION_DATA_SIZE, BATCH_SIZE, input_size = INPUT_SIZE[:2],material_path=SOURCE_PATH,
  label_path=LABEL_PATH, background_images_path=BACKGROUND_IMAGEES_PATH,
  use_multiprocessing=True, workers = 8
)
train_dataset = tf.data.Dataset.from_generator(
  lambda: train_sequence, 
  output_signature=(
    tf.TensorSpec(shape=(BATCH_SIZE, INPUT_SIZE[0], INPUT_SIZE[1], INPUT_SIZE[2]), dtype=tf.float32),
    {
      "roi": tf.TensorSpec(shape=(BATCH_SIZE, INPUT_SIZE[0], INPUT_SIZE[1]), dtype=tf.bool),
      "centroid": tf.TensorSpec(shape=(BATCH_SIZE * 16, 2), dtype=tf.float32),
      "detection": tf.TensorSpec(shape=(BATCH_SIZE * 16, 4), dtype=tf.float32),
      "segmentation": tf.TensorSpec(shape=(BATCH_SIZE * 16, INPUT_SIZE[0]//4, INPUT_SIZE[1]//4), dtype=tf.bool),
      "classification": tf.TensorSpec(shape=(BATCH_SIZE * 16), dtype=tf.uint32)
    }
  )
)

In [8]:
model.compile()

In [9]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
  filepath="/workspaces/dev/models/R-CNN/checkpoints/pill_net.keras",
  monitor="val_loss",
  save_best_only=True,
  save_weights_only=False,
  verbose = 1
)

In [10]:
early_stopping_cb = keras.callbacks.EarlyStopping(
  monitor="val_loss",
  patience=10,
  restore_best_weights=True,
  verbose = 1
)

In [ ]:
model.fit(
  train_sequence,
  validation_data=validation_sequence,
  epochs=100,
  callbacks=[checkpoint_cb, early_stopping_cb],
  verbose=1
)

Epoch 1/100


I0000 00:00:1741582732.224391   39447 service.cc:148] XLA service 0x7f79cc00bd40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741582732.224436   39447 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2025-03-10 04:58:52.379757: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1741582732.650928   39447 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert/Assert
W0000 00:00:1741582732.651006   39447 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_1/Assert
W0000 00:00:1741582732.652194   39447 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_2/Assert
W0000 00:00:1741582732.652259   39447 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while

   1/3750 ━━━━━━━━━━━━━━━━━━━━ 21:01:22 20s/step - centroid_loss: 0.0042 - classification_loss: 3.9182 - detection_loss: 0.0131 - roi_loss: 0.6512 - segmentation_loss: 0.6616 - total_loss: 5.2482

I0000 00:00:1741582745.637935   39447 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1741582746.159396   39446 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert/Assert
W0000 00:00:1741582746.159437   39446 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_1/Assert
W0000 00:00:1741582746.159537   39446 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_2/Assert
W0000 00:00:1741582746.159542   39446 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_3/Assert
W0000 00:00:1741582746.159557   39446 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_4/Assert
W0000 00:00:1741582746.159561   39446 assert_op.cc:38] Ignoring Assert operator pill_net_1/map/while/crop_to_bounding_box/Assert_5/Assert
W0000 00:00:174158

  39/3750 ━━━━━━━━━━━━━━━━━━━━ 6:06:59 6s/step - centroid_loss: 0.0033 - classification_loss: 3.9164 - detection_loss: 0.0183 - roi_loss: 0.5081 - segmentation_loss: 0.3976 - total_loss: 4.8438